# Formulation avec cycles de taille 3

In [ ]:
function get_cycles_3(graph)
    """
    Construit les données utiles du graphe restreint aux cycles de taille 3.
    
    Entrée : 
        - graph : array - Liste des caractéristiques du graphe.
    
    Sortie :
        - cycles       : array - Liste des cycles de taille <= 3
        - C_a          : array - Liste des identifiants des cycles contenant l'arc a
        - C_v          : array - Liste des identifiants des cycles passant par le sommet v
        - U            : array - Liste des coûts de chaque cycle
        - collect(A_3) : array - Liste des arcs concernés par les cycles
        - collect(V_3) : array - Liste des sommets concernés par les cycles
    """
    
    # Récupération des propriétés du graphe
    n = graph[1]
    A = graph[2]
    preds = graph[4]
    succ = graph[5]
    
    # Initialisation des listes 
    cycles = []                           # Liste des cycles de taille <= 3
    visited = []                          # Liste des cycles déjà ajoutés dans cycles
    C_a = [ [] for i in 1:n, j in 1:n ]   # Liste des cycles contenant l'arc "(i,j)" (pour tout arc)
    C_v = [ [] for i in 1:n]              # Liste des cycles passant par le sommet "i" (pour tout sommet)
    U = []                                # Coût du cycle ajouté
    A_3 = Set()                           # Ensemble des arcs impliqués dans les cycles de taille <= 3
    V_3 = Set()                           # Ensemble des sommets impliqués dans les cycles de taille <= 3
    
    # On parcourt les sommets du graphe
    for (i,j) in A
        if i > j 
            # Pour casser la symétrie
            continue
        end
        # Intersection des successeurs de j et des prédecesseurs de i
        sommets = intersect(succ[j],preds[i])
        
        # Récupération des cycles de taille 3
        for s in sommets
            # Si le cycle (i,j,s) n'a pas encore été visité
            if !(Set((i,s,j)) in visited)
                # Ajout du cycle
                push!(cycles, [(i,j),(j,s),(s,i)])
                
                # Mise à jour de A_3
                push!(A_3,(i,j))
                push!(A_3,(j,s))
                push!(A_3,(s,i))
                            
                # Mise à jour de V_3
                push!(V_3,i)
                push!(V_3,j)
                push!(V_3,s)
                
                # Coût du cycle
                push!(U,3)
                
                # Mise à jour de visited
                push!(visited,Set((i,s,j)))
                
                # Mises à jour de C_a et C_v
                push!(C_a[i,j],length(cycles))
                push!(C_a[j,s],length(cycles))
                push!(C_a[s,i],length(cycles))
                            
                push!(C_v[i], length(cycles))
                push!(C_v[j], length(cycles))
                push!(C_v[s], length(cycles))
            end
        end
        
        # Si l'arc (j,i) existe, il existe un cycle de taille 2
        if (j,i) in A 
            # Ajout du cycle dans la liste C_2
            push!(cycles,[(i,j),(j,i)])
                
            # Mises à jour de A_3 et V_3
            push!(A_3,(i,j))
            push!(A_3,(j,i))
                        
            push!(V_3,i)
            push!(V_3,j)
            
            # Ajout du coût du cycle
            push!(U,2)
                
            # Mises à jour de C_v et C_a
            push!(C_v[i], length(cycles))
            push!(C_v[j], length(cycles))

            push!(C_a[i,j], length(cycles))
            push!(C_a[j,i], length(cycles))
        end
            
    end
    
    return cycles, C_a, C_v, U, collect(A_3), collect(V_3)
    
end

In [1]:
function uncertain_model_3(A_3,V_3,C,C_a,C_v,U, S, B, logs,relax=false,t_max=120)
    """
    Crée et optimise le programme linéaire avec cycles de taille <= 3.

    Entrées : 
        - A_3   : array   - Liste des arcs concernés par les cycles de taille <= 3
        - V_3   : array   - Liste des sommets concernés par les cycles de taille <= 3
        - C     : array   - Liste des cycles de taille <= 3
        - C_a   : array   - Liste des identifiants des cycles de taille <= 3
        - C_v   : array   - Liste des identifiants des cycles passant par chaque sommet 
        - U     : array   - Liste des coûts de chaque cycle
        - S     : array   - Liste des scénarios
        - B     : Integer - Budget (nombre de tests maximum)
        - logs  : Boolean - Booléen pour l'affichage de la solution
        - relax : Boolean - Booléen pour relâcher la contrainte d'intégrité
        - t_max : Integer - Temps maximum accordé pour la résolution du problème
    
    Sortie : 
        - objective_value(model) : Float      - Valeur optimale du modèle résolu 
        - model                  : JuMP Model - Modèle du problème
        - timer                  : Float      - Temps de calcul
    """
    # Vecteur des sommets et vecteur des hôpitaux
    N = Array(1:length(S))

    # Création du modèle
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)

    # Variables
    # X[(i,j)] = 1 ssi on réalise le test croisé sur l'arc (i,j)
    @variable(model, x[A_3], Bin);
    # z[c,s] = 1 ssi on choisit le cycle d'indice c dans le scénario s
    if relax
        @variable(model, z[1:length(C),N]>=0);
    else
        @variable(model, z[1:length(C),N], Bin);
    end

    # Fonction objectif: On souhaite maximiser la moyenne empirique du nombre de transplantation
    @objective(model, Max, 1/length(S)*sum(U[c]*z[c,s] for c in 1:length(C) for s in N));
    
    # Contraintes
    ## On réalise au plus B tests croisés (B = budget de tests)
    @constraint(model,c1, sum(x[a] for a in A_3) <= B)
    
    ## On ne peut choisir un arc que si le test croisé a été réalisé et a été réussi pour cet arc
    @constraint(model,c2[a in A_3, s in N],
        sum(z[c,s] for c in C_a[a[1],a[2]]) <= x[a] * S[s][a[1],a[2]])
    
    ## Chaque sommet peut être parcouru par au plus un cycle, dans le scénario s
    @constraint(model,c4[v in V_3, s in N], sum(z[c,s] for c in C_v[v]) <= 1)
    
    # Ajout du time_limit donné
    set_time_limit_sec(model, t_max)
    
    # Résolution du problème
    timer = @timed optimize!(model);  
    
    # Affichage si time_limit atteint
    if termination_status(model) == TIME_LIMIT
        println("Attention : time_limit atteint")
    end
    # Affichage de la solution si souhaité
    if logs 
        println("***** Uncertain *****");
        # Statistiques de la solution
        println("Statut de la solution = ", termination_status(model));
        println("Valeur optimale = ", objective_value(model));
        println("Temps = ", timer[2])
    end
            
    return objective_value(model),model,timer;
end

LoadError: LoadError: UndefVarError: `@variable` not defined
in expression starting at In[1]:36

In [ ]:
function pb_recours_3(A_3,V_3,C, C_a, C_v, U, S, logs, X, relax=false)
    """
    Crée et optimise le programme linéaire avec cycles de taille <= 3.

    Entrées : 
        - A_3   : array          - Liste des arcs concernés par les cycles de taille <= 3
        - V_3   : array          - Liste des sommets concernés par les cycles de taille <= 3
        - C     : array          - Liste des cycles de taille <= 3
        - C_a   : array          - Liste des identifiants des cycles de taille <= 3
        - C_v   : array          - Liste des identifiants des cycles passant par chaque sommet 
        - U     : array          - Liste des coûts de chaque cycle
        - S     : array          - Liste des scénarios
        - logs  : Boolean        - Booléen pour l'affichage de la solution
        - X     : DenseAxisArray - Liste des tests croisés réalisés
        - relax : Boolean        - Booléen pour relâcher la contrainte d'intégrité
    
    Sortie : 
        - objective_value(model) : Float      - Valeur optimale du modèle résolu 
        - model                  : JuMP Model - Modèle du problème
        - timer                  : Float      - Temps de calcul
    """
   
    # Vecteur des sommets et vecteur des hôpitaux
    N = Array(1:length(S))

    # Création du modèle
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)

    # Variables
    # z[c,s] = 1 ssi on choisit le cycle d'indice c dans le scénario s
    @variable(model, z[1:length(C),N], Bin);
    
    # Fonction objectif: On souhaite maximiser la moyenne empirique du nombre de transplantation
    @objective(model, Max, 1/length(S)*sum(U[c]*z[c,s] for c in 1:length(C) for s in N));
    
    # Contraintes

    ## On ne peut choisir un arc que si le test croisé a été réalisé et a été réussi pour cet arc
    @constraint(model,c2[a in A_3, s in N],
        sum(z[c,s] for c in C_a[a[1],a[2]]) <= X[a] * S[s][a[1],a[2]])
    
    ## Chaque sommet peut être parcouru par au plus un cycle, dans le scénario s
    @constraint(model,c4[v in V_3, s in N], sum(z[c,s] for c in C_v[v]) <= 1)
    
    # On relâche l'intégrité si souhaité
    if relax
        # Si l'on souhaite résoudre le problème avec contrainte d'intégrité
        relax_integrality(model)
    end 
    
    # Résolution du problème
    timer = @timed optimize!(model);  

    # Affichage de la solution si souhaité
    if logs 
        println("***** Uncertain *****");
        # Statistiques de la solution
        println("Statut de la solution = ", termination_status(model));
        println("Valeur optimale = ", objective_value(model));
        println("Temps = ", timer[2])
    end
            
    return objective_value(model),model,timer;
end

In [ ]:
function compute_solution_uncertain(model)
    """
    Crée et affiche les cycles solutions rendu par le modèle fourni.
    
    Entrées : 
        - model : JuMP Model - Modèle résolu du problème
    
    Sorties : 
        - arcs : Array - Cycles solutions
    """
    # Récupération des valeurs optimales des variables et de la valeur optimale
    X = value.(model[:x])

    # Récupération des arcs solution
    arcs = []
    for (i,j) in axes(X)[1]
        if abs(X[(i,j)]-1) < 1e-6
            push!(arcs,(i,j))
        end
    end
        
    println("Tests croisés à réaliser : ")
    for arc in arcs
        print("  - ")
        println(arc) 
    end
    
    return arcs
end

In [ ]:
println("Fonctions de résolution du problème de taille 3 importées avec succès.")